In [1]:
from pathlib import Path
import numpy as np
import math

import georeferencing, crs_tools

In [2]:
# Paths
lcf_path = Path.cwd() / 'ExampleImages' / 'Juvika_June2022_Pika_L_12.lcf'
times_path = lcf_path.parent / (lcf_path.stem + '.bil.times')

In [3]:
# Load LCF data
lcf_data = georeferencing.read_lcf_file(lcf_path)
print(lcf_data)


{'time': array([0.00000000e+00, 5.00011444e-03, 9.99999046e-03, ...,
       1.83250000e+01, 1.83300002e+01, 1.83350000e+01]), 'roll': array([-0.01800563, -0.01799441, -0.01794705, ..., -0.00483355,
       -0.00484231, -0.00485931]), 'pitch': array([-0.11502507, -0.11497966, -0.11493181, ..., -0.0723944 ,
       -0.07240727, -0.07242776]), 'yaw': array([2.21383309, 2.2133956 , 2.21344376, ..., 2.12589884, 2.1260078 ,
       2.12592864]), 'longitude': array([15.0050441 , 15.00504447, 15.00504483, ..., 15.00621286,
       15.00621319, 15.00621352]), 'latitude': array([67.21564225, 67.2156421 , 67.21564197, ..., 67.21515659,
       67.21515646, 67.21515633]), 'altitude': array([38.51876425, 38.513728  , 38.51277217, ..., 37.69600152,
       37.69522698, 37.69445147])}


In [4]:
image_times = georeferencing.read_times_file(times_path)
print(image_times)

[0.0000000e+00 9.1750000e-03 1.8349000e-02 ... 1.8321089e+01 1.8330263e+01
 1.8339437e+01]


In [27]:
# Calculate transform parameters
framerate = 1/np.mean(np.diff(image_times))
# framerate = 200
ground_altitude = -10
pitch_offset = np.mean(lcf_data['pitch'])
# pitch_offset = 0.1
roll_offset = np.mean(lcf_data['roll'])
# roll_offset = -0.1
affine_transform_parameters, utm_epsg = georeferencing.calculate_pushbroom_imager_transform(
    lcf_data['time'],
    lcf_data['longitude'],
    lcf_data['latitude'],
    lcf_data['altitude'],
    framerate,
    use_world_file_ordering=True,
    ground_altitude = ground_altitude,
    pitch_offset = pitch_offset,
    roll_offset = roll_offset)
print(affine_transform_parameters)
print(f'{utm_epsg=}')

(0.025746833032614692, 0.024024646518708413, 0.025290239927096874, -0.027103149436586788, 500204.3131578073, 7455411.765404667)
utm_epsg=32633


In [28]:
# Write world file
world_file_path = lcf_path.parent / (lcf_path.stem + '.wld')
np.savetxt(world_file_path,affine_transform_parameters)

In [49]:
def world_file_from_lcf_times_files(lcf_file_path,times_file_path,world_file_path,**kwargs):
    """ Calculate world file (affine transformation) for raster image based on .lcf and .times files 
    
    # Arguments:
    lcf_file_path:
        Path to .lcf file, containing camera IMU and GNSS information
    times_file_path:
        Path to .times file. 
    world_file_path:
        Path for output world file. Should have file extension corresponding to
        image format, e.g. 'pgw' for PNG files, or, alternatively,
        just 'wld' (accepted by GDAL and QGIS). 
         
    # Keyword arguments:
    Keyword parameters are passed on to calculate_pushbroom_imager_transform().
    Note: pitch_offset and roll_offset are estimated as mean values from LCF files,
    and can (currently) not be set manually.  All other keyword arguments to the 
    function (e.g. ground_altitude) can be passed in. 
    """

    lcf_data = georeferencing.read_lcf_file(lcf_file_path)
    image_times = georeferencing.read_times_file(times_file_path)
    framerate = 1/np.mean(np.diff(image_times))

    # Calculate transform parameters
    affine_transform_parameters, utm_epsg = georeferencing.calculate_pushbroom_imager_transform(
        lcf_data['time'],
        lcf_data['longitude'],
        lcf_data['latitude'],
        lcf_data['altitude'],
        framerate,
        use_world_file_ordering=True,
        pitch_offset = np.mean(lcf_data['pitch']),
        roll_offset = np.mean(lcf_data['roll']),
        **kwargs)

    # Save to file
    np.savetxt(world_file_path,affine_transform_parameters)

In [52]:
world_file_from_lcf_times_files(lcf_path,times_path,world_file_path,ground_altitude=-10)

In [46]:
def my_func(a,b='hello',**kwargs):
    print(f'{a=}, {b=}')
    print(kwargs)
    my_func2(**kwargs)

def my_func2(c=42,**kwargs):
    print(f'{c=}')

In [47]:
my_func(4,b='satan',d='world',c=14)

a=4, b='satan'
{'d': 'world', 'c': 14}
c=14
